# install package

In [2]:
pip install transformers

  Using cached transformers-3.5.1-py3-none-any.whl (1.3 MB)
  Using cached sentencepiece-0.1.91-cp37-cp37m-win_amd64.whl (1.2 MB)
Processing c:\users\acer\appdata\local\pip\cache\wheels\69\09\d1\bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0\sacremoses-0.0.43-py3-none-any.whl
  Using cached tokenizers-0.9.3-cp37-cp37m-win_amd64.whl (1.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\acer\anaconda3

  added / updated specs:
    - cudatoolkit=10.2
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py37haa95532_0         2.9 MB
    cudatoolkit-10.2.89        |       h74a9793_1       317.2 MB
    libuv-1.40.0               |       he774522_0         255 KB
    ninja-1.10.1               |   py37h7ef1ec2_0         249 KB
    pytorch-1.7.0              |py3.7_cuda102_cudnn7_0       767.6 MB  pytorch
    torchaudio-0.7.0           |             py37         2.7 MB  pytorch
    torchvision-0.8.1          |       py37_cu102         7.2 MB  pytorch
    typing_extensions-3.7.4.3  |             py_0          28 KB
    ------------------------------------------------------------
                              

# prepare data

In [5]:
import pandas as pd
df = pd.read_pickle(r'data.pickle')

In [6]:
df = pd.DataFrame(df)
df.head()

,id,question,answer,labeller,emo_class,seven_emo,intent,topic,keyword_vec,similarity,reviewed,created_at,updated_at,url
0,1,ชิว,ก็ดีแล้ว นอนชิว นั่งชิว ขนาดกินยังชิว :),None,pos_de,relaxed,chitchat,general,"0.6564798951148987,-1.8736761808395386,0.49881...",1.0,True,2019-06-03T13:52:59.000+07:00,2019-08-04T08:03:10.000+07:00,http://chitchat.tunejai.com/chats/1.json
1,2,ไม่นอนในเวลางานสิ,ของีบนิดนึงไม่ได้หรอ,None,neu,neutral,chitchat,general,"0.5782301053404808,0.3987545693914096,0.271282...",1.0,True,2019-06-03T13:52:59.000+07:00,2019-08-04T08:04:00.000+07:00,http://chitchat.tunejai.com/chats/2.json
2,3,อืมา,🙂,None,neu,neutral,chitchat,general,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,True,2019-06-03T13:52:59.000+07:00,2019-08-04T08:04:23.000+07:00,http://chitchat.tunejai.com/chats/3.json
3,4,หยิบยังไงอะ,เอามือหยิบสิ,None,neu,neutral,chitchat,general,"0.038666918873786926,-0.18005990283563733,-0.1...",1.0,True,2019-06-03T13:52:59.000+07:00,2019-08-04T08:04:50.000+07:00,http://chitchat.tunejai.com/chats/4.json
4,5,ระหว่าง sumsung กับ iphone ใช้อะไรดี,เธอว่าไงก็ว่างั้นละ มันแล้วแต่คนชอบ,None,neu,neutral,question,general,"0.3020523651079698,0.10027491030367938,-0.2324...",1.0,True,2019-06-03T13:52:59.000+07:00,2019-08-04T08:05:24.000+07:00,http://chitchat.tunejai.com/chats/5.json


In [11]:
features_and_label = df[['question','seven_emo']]

#ค่า label ที่เป็น None ตัดทิ้งไปได้เลย ต่อให้มี question แต่ไม่รู้เฉลยก็ไร้ประโยชน์
features_and_label = features_and_label.dropna()

#มีค่าความรู้สึกนึงที่น้อยที่สุดอยู่ที่ 1080 ตัว ซึ่ง Mink เขียนโค้ดหาให้แล้ว ดังนั้นเราจึง sampling ความรู้สึกอื่นๆ สุ่มตัวอย่างมาแค่ 1080 ตัวเท่าๆ กัน
angry = features_and_label.loc[features_and_label['seven_emo']=='angry'].sample(1028)
bored = features_and_label.loc[features_and_label['seven_emo']=='bored'].sample(1028)
depressed = features_and_label.loc[features_and_label['seven_emo']=='depressed'].sample(1028)
happy = features_and_label.loc[features_and_label['seven_emo']=='happy'].sample(1028)
neutral = features_and_label.loc[features_and_label['seven_emo']=='neutral'].sample(1028)
relaxed = features_and_label.loc[features_and_label['seven_emo']=='relaxed'].sample(1028)
stressed = features_and_label.loc[features_and_label['seven_emo']=='stressed'].sample(1028)

features_and_label = angry
features_and_label = features_and_label.append(bored)
features_and_label = features_and_label.append(depressed)
features_and_label = features_and_label.append(happy)
features_and_label = features_and_label.append(neutral)
features_and_label = features_and_label.append(relaxed)
features_and_label = features_and_label.append(stressed)

In [35]:
X = features_and_label['question'] #features
y = features_and_label['seven_emo'] #labels

#แปลง seven_emo ให้กลายเป็นตัวเลข 0-6 ก่อน
new_y = []
for i in y:
    if i == 'angry':
        new_y.append(0)
    elif i == 'bored':
        new_y.append(1)
    elif i == 'depressed':
        new_y.append(2)
    elif i == 'happy':
        new_y.append(3)
    elif i == 'neutral':
        new_y.append(4)
    elif i == 'relaxed':
        new_y.append(5)
    else:
        new_y.append(6)
y = new_y

from sklearn.model_selection import train_test_split
#แบ่ง train กับ test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#แบ่ง validation ออกมาจาก train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [31]:
print("X_train = ",len(X_train),",y_train = ",len(y_train),"\n","X_val = ",len(X_val),",y_val = ",len(y_val))

X_train =  3525 ,y_train =  3525 
 X_val =  1512 ,y_val =  1512


In [18]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

In [7]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-multilingual-cased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

In [10]:
#ลองทดสอบดูหน่อย
text = ["และในวันที่ฝนตกฉันจะจำตลอดไป...ถึงวันที่เคยมีเธอเพระาตอนนี้ไม่มีอีกต่อไป","ต่อจากนี้ ในคำคืนที่ฟ้าร้องคงต้องเหงา"]

sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)
print(sent_id)

{'input_ids': [[101, 15493, 75890, 37840, 111421, 16000, 30011, 18427, 111413, 23687, 46856, 22598, 46856, 55593, 30011, 20507, 33178, 22123, 111435, 49292, 119, 119, 119, 64175, 37840, 111431, 31256, 80785, 18260, 102058, 111431, 39901, 22765, 22598, 17344, 30011, 45629, 80420, 111435, 17405, 84307, 18260, 33178, 18260, 18427, 30011, 98455, 111435, 49292, 102], [101, 1413, 98455, 46856, 101983, 80420, 46301, 31256, 55593, 31256, 111429, 16000, 18203, 72245, 43102, 22765, 75784, 31256, 19197, 30011, 75784, 111431, 111424, 19197, 17344, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [37]:
max_seq_len = 7

# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length = max_seq_len,
    padding=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    X_val.tolist(),
    max_length = max_seq_len,
    padding=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length = max_seq_len,
    padding=True,
    truncation=True,
    return_token_type_ids=False
)

In [40]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(y_train)

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(y_val)

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(y_test)

In [41]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [55]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [56]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,7)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [57]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [58]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

In [59]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(y_train), y_train)

print(class_wts)

[1.01526498 1.01118761 1.02979842 0.95373377 1.00513259 0.98933483
 0.99914966]


In [60]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 20

In [61]:
weights

tensor([1.0153, 1.0112, 1.0298, 0.9537, 1.0051, 0.9893, 0.9991],
       device='cuda:0')

In [62]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [49]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [70]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 20
  Batch    50  of    111.
  Batch   100  of    111.

Evaluating...

Training Loss: 1.867
Validation Loss: 1.849

 Epoch 2 / 20
  Batch    50  of    111.
  Batch   100  of    111.

Evaluating...

Training Loss: 1.864
Validation Loss: 1.868

 Epoch 3 / 20
  Batch    50  of    111.
  Batch   100  of    111.

Evaluating...

Training Loss: 1.856
Validation Loss: 1.841

 Epoch 4 / 20
  Batch    50  of    111.
  Batch   100  of    111.

Evaluating...

Training Loss: 1.858
Validation Loss: 1.840

 Epoch 5 / 20
  Batch    50  of    111.
  Batch   100  of    111.

Evaluating...

Training Loss: 1.849
Validation Loss: 1.848

 Epoch 6 / 20
  Batch    50  of    111.
  Batch   100  of    111.

Evaluating...

Training Loss: 1.851
Validation Loss: 1.863

 Epoch 7 / 20
  Batch    50  of    111.
  Batch   100  of    111.

Evaluating...

Training Loss: 1.853
Validation Loss: 1.849

 Epoch 8 / 20
  Batch    50  of    111.
  Batch   100  of    111.

Evaluating...

Training Loss: 1.852
Validat

In [71]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [72]:
# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

In [73]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.30      0.28      0.29       319
           1       0.31      0.41      0.35       316
           2       0.28      0.44      0.35       320
           3       0.26      0.25      0.26       278
           4       0.24      0.22      0.23       304
           5       0.33      0.31      0.32       308
           6       0.36      0.14      0.20       314

    accuracy                           0.29      2159
   macro avg       0.30      0.29      0.28      2159
weighted avg       0.30      0.29      0.29      2159

